In [1]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.datasets import fetch_openml
from sklearn.preprocessing import OneHotEncoder

In [2]:
class LogisticRegression:
    
    def __init__(self, lr=0.01, num_iter=100000, fit_intercept=True, verbose=False):
        self.lr = lr
        self.num_iter = num_iter
        self.fit_intercept = fit_intercept
        self.verbose = verbose
    
    def __add_intercept(self, X):
        intercept = np.ones((X.shape[0], 1))
        return np.concatenate((intercept, X), axis=1)
    
    def __sigmoid(self, z):
        return 1 / (1 + np.exp(-z))
    
    def __loss(self, h, y):
        return (-y * np.log(h) - (1 - y) * np.log(1 - h)).mean()
    
    def fit(self, X, y):
        if self.fit_intercept:
            X = self.__add_intercept(X)
        
        # weights initialization
        self.theta = np.zeros(X.shape[1])
        
        for i in range(self.num_iter):
            z = np.dot(X, self.theta)
            h = self.__sigmoid(z)
            gradient = np.dot(X.T, (h - y)) / y.size
            self.theta -= self.lr * gradient
            
            if(self.verbose == True and i % 10000 == 0):
                z = np.dot(X, self.theta)
                h = self.__sigmoid(z)
                print(f'loss: {self.__loss(h, y)} \t')
    
    def predict_prob(self, X):
        if self.fit_intercept:
            X = self.__add_intercept(X)
    
        return self.__sigmoid(np.dot(X, self.theta))
    
    def predict(self, X, threshold):
        return self.predict_prob(X) >= threshold
    

In [16]:
X, y = fetch_openml('mnist_784', return_X_y=True)
y = y.astype(int)
X /= 255

In [5]:
y[:10]

array([5, 0, 4, 1, 9, 2, 1, 3, 1, 4])

In [6]:
ohe = OneHotEncoder(sparse=False, categories=[range(10)])
y_ohe = ohe.fit_transform(y.reshape(-1,1))
X_train, X_test, y_train, t_test = train_test_split(X, y_ohe)

In [7]:
y_ohe[:10]

array([[0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
       [0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0., 0., 0., 0.]])

In [8]:
model = LogisticRegression(lr=0.1, num_iter=1000)
%time model.fit(X_train, y_train[:,6])

/home/ram/anaconda3/envs/tfenv/lib/python3.6/site-packages/ipykernel_launcher.py:14: RuntimeWarning: overflow encountered in exp
  


CPU times: user 7min 34s, sys: 47.2 s, total: 8min 21s
Wall time: 1min 7s


In [9]:
preds = model.predict(X, threshold=0.5)

/home/ram/anaconda3/envs/tfenv/lib/python3.6/site-packages/ipykernel_launcher.py:14: RuntimeWarning: overflow encountered in exp
  


In [11]:
preds[:10]

array([False, False, False, False, False, False, False, False, False,
       False])

In [10]:
(y_ohe[:,6] == preds.astype(int)).sum() / y_ohe.shape[0]

0.9865571428571429

In [ ]:
class MultinomialLogisticRegression():
    
    def __init__(self, lr=0.01, num_iter=100000, fit_intercept=True, verbose=False):
        self.lr = lr
        self.num_iter = num_iter
        self.fit_intercept = fit_intercept
        self.verbose = verbose
    
    def __add_intercept(self, X):
        intercept = np.ones((X.shape[0], 1))
        return np.concatenate((intercept, X), axis=1)
    
    def __softmax(self, z):
        e_z = np.exp(-z)
        return np.divide(e_z, np.sum(e_z, axis=1)[:,None], out=np.zeros_like(e_z), where=e_z!=0)

    def __cross_entropy(self, h, y):
        m = y.shape[0]
        y = y == 1
        # We use multidimensional array indexing to extract 
        # softmax probability of the correct label for each sample.
        # Refer to https://docs.scipy.org/doc/numpy/user/basics.indexing.html#indexing-multi-dimensional-arrays for understanding multidimensional array indexing.
        log_likelihood = -np.log(h[y])
        loss = np.sum(log_likelihood) / m
        return loss
    
    def fit(self, X, y):
        if self.fit_intercept:
            X = self.__add_intercept(X)
        
        # weights initialization
        self.thetas = np.ones((X.shape[1], y.shape[1]))
        
        for i in range(self.num_iter):
            z = np.dot(X, self.thetas)
            h = self.__softmax(z)

            gradient = np.dot(X.T, (h - y)) / y.size
            self.thetas -= self.lr * gradient
            
            if(self.verbose == True and i % 100 == 0):
                z = np.dot(X, self.thetas)
                h = self.__softmax(z)
                print(f'loss: {self.__cross_entropy(h, y)} \t')
        
    def predict_prob(self, X):
        if self.fit_intercept:
            X = self.__add_intercept(X)
        return self.__softmax(np.dot(X, self.thetas))
    
    def predict(self, X):
        return np.argmax(self.predict_prob(X), axis=1)
    
    

In [ ]:
y[:10]

In [ ]:
model = MultinomialLogisticRegression(lr=0.1, num_iter=1000, verbose=True)
%time model.fit(X_train, y_train)

In [ ]:
preds = model.predict_prob(X)[:10]

In [ ]:
preds

In [ ]:
model.thetas

In [ ]:
preds = model.predict(X)
preds.sum()

In [ ]:
y[:10]

In [ ]:
intercept = np.ones((X.shape[0], 1))
xi = np.concatenate((intercept, X), axis=1)
print((==0).sum())

In [ ]:
def softmax(z):
    e_z = np.exp(-z)
    print(np.sum(e_z))
    return np.divide(e_z, np.sum(e_z, axis=1)[:,None], out=np.zeros_like(e_z), where=e_z!=0)

In [ ]:
np.dot(xi, model.thetas)

In [ ]:
# accuracy
sum() / preds.shape

In [ ]:
np.ones((3,5)).sum(axis=1)